In [57]:
import random
from datetime import timedelta
from utilities import generate_random_date, generate_licence_plate

In [58]:
car_brands = {
  "Bmw": ["Series 3"],
  "Toyota": ["Corolla", "Land Cruiser", "Hilux", "Rav"],
  "Skoda": ["Superb"],
  "Kia": ["Stinger GT", "Ceed 3"],
  "Volkswagen": ["Crafter", "Passat"],
  "Hyundai": ["Ionia", "Tucson"]
}

motorbike_brands = {
  "Triumph": ["Tiger 1050"],
  "Kawasaki": ["Versys 1000"],
  "BMW": ["R 1250 RT"],
}

In [59]:
from numpy import loadtxt
names_male = loadtxt("./data/supporting/names/polish_male_firstnames.txt", dtype='str')
names_female = loadtxt("./data/supporting/names/polish_female_firstnames.txt", dtype='str')
surnames = loadtxt("./data/supporting/names/polish_surnames.txt", dtype='str')
genders = ["female", "male"]
ranks = ["officer", "normal"]

print(len(names_male))
print(len(names_female))
print(len(surnames))

300
300
314549


# Define object classes

In [60]:
class Vehicle:
  def __init__(self, license_plate_number, vehicle_type, brand, model, manufacturing_year):
    
    self.license_plate_number = license_plate_number
    
    # "car" or "motorbike"
    self.vehicle_type = vehicle_type
    self.brand = brand
    self.model = model
    self.manufacturing_year = manufacturing_year
    
    # meters per second
    self.max_speed = 100
    self.last_inspection = manufacturing_year

In [61]:
class Officer:
  def __init__(self, officer_id, first_name, last_name, rank, gender, date_of_birth, hire_date, email, phone_number):
    
    self.officer_id = officer_id
    self.first_name = first_name
    self.last_name = last_name
    self.rank = rank
    self.gender = gender
    self.date_of_birth = date_of_birth
    self.hire_date = hire_date
    self.email = email
    self.phone_number = phone_number

In [62]:
class Victim:
  def __init__(self, victim_id, first_name, last_name, gender, date_of_birth):
    
    self.victim_id = victim_id
    self.first_name = first_name
    self.last_name = last_name
    self.gender = gender
    self.date_of_birth = date_of_birth

In [63]:
class City:
  def __init__(self, city_name, city_districts):
    
    self.city_name = city_name
    self.city_districts = city_districts

# Define Generator

In [64]:
INITIAL_COUNTS = {
  "officers": 100,
  "cars": 70,
  "motorbikes": 30
}

class Generator:
  def __init__(self, initial_counts, vehicle_pos_regestration_timestep, incidents_per_hour):
    
    self.initial_counts = initial_counts
    # In seconds
    self.vehicle_pos_regestration_timestep = vehicle_pos_regestration_timestep
    self.incidents_per_hour = incidents_per_hour
    
    
    self.cars = [None] * initial_counts["cars"]
    self.motorbikes = [None] * initial_counts["motorbikes"]
    self.officers = [None] * initial_counts["officers"]
    
  def generate_initial_data(self, initial_date):
    
    def create_officer(id):
      MIN_OFFICER_AGE = 21
      MAX_OFFICER_AGE = 70
      MAX_OFFICER_HIRE_TIME = 35

      gender = random.choice(genders)
      last_name = random.choice(surnames)
      
      hire_date = generate_random_date(initial_date - timedelta(days=MAX_OFFICER_HIRE_TIME*365), initial_date)
      
      date_of_birth = generate_random_date(initial_date - timedelta(days=MAX_OFFICER_AGE*365), hire_date - timedelta(days=MIN_OFFICER_AGE*365))
      
      rank = random.choice(ranks)

      if gender == "male":
        first_name = random.choice(names_male)
      else:
        first_name = random.choice(names_female)
      
      officer = Officer(id, first_name, last_name, rank, gender, date_of_birth, hire_date, "officer{}@police.org.pl".format(id), 111111111)
      # print(date_of_birth, hire_date)
      
      return officer
    
    def create_car():
      
      OLDEST_CAR_YEAR = 10
      EARLIEST_CAR_YEAR = 1
      
      license_plate_number = generate_licence_plate()
      brand = random.choice(list(car_brands.keys()))
      model = random.choice(car_brands[brand])
      manufacturing_year = generate_random_date(initial_date - timedelta(days=OLDEST_CAR_YEAR*365), initial_date - timedelta(days=EARLIEST_CAR_YEAR*365))
      
      car = Vehicle(license_plate_number, "car", brand, model, manufacturing_year)
      # print(manufacturing_year, brand, model, license_plate_number)
      
      return car
      
    def create_motorbike():
      OLDEST_CAR_YEAR = 10
      EARLIEST_CAR_YEAR = 1
      
      license_plate_number = generate_licence_plate()
      brand = random.choice(list(motorbike_brands.keys()))
      model = random.choice(motorbike_brands[brand])
      manufacturing_year = generate_random_date(initial_date - timedelta(days=OLDEST_CAR_YEAR*365), initial_date - timedelta(days=EARLIEST_CAR_YEAR*365))
      
      motorbike = Vehicle(license_plate_number, "motorbike", brand, model, manufacturing_year)
      print(manufacturing_year, brand, model, license_plate_number)
      
      return motorbike
    
    
    create_officer(1)
    
    for i in range(1, self.initial_counts["officers"]):
      officer = create_officer(i)
      self.officers[i] = officer
      
    for i in range(1, self.initial_counts["cars"]):
      car = create_car()
      self.cars[i] = car
      
    for i in range(1, self.initial_counts["motorbikes"]):
      motorbike = create_motorbike()
      self.motorbikes[i] = motorbike
      
    
  def simulate(self, from_time, to_time):
    time_diff = to_time - from_time
    iterations = time_diff.total_seconds() / self.vehicle_pos_regestration_timestep
    print(time_diff.total_seconds(), iterations)
    for i in range(1, int(iterations)):
      print(i)

In [65]:
generator = Generator(INITIAL_COUNTS, 300, 15)

from datetime import date
startdate = date(2023, 1, 1)
enddate = date(2023, 12, 31)

generator.generate_initial_data(enddate)
# generator.simulate(startdate, enddate)

2018-07-29 Kawasaki Versys 1000 BNF8050
2019-08-22 BMW R 1250 RT ZYY6608
2020-05-25 BMW R 1250 RT MAZ7396
2017-12-27 Kawasaki Versys 1000 UVV3389
2021-03-25 Kawasaki Versys 1000 KBU2544
2021-03-18 BMW R 1250 RT ZQO7302
2022-09-13 BMW R 1250 RT BJT5007
2014-11-02 Triumph Tiger 1050 XJB4662
2014-01-10 Triumph Tiger 1050 ZEP5721
2020-08-21 BMW R 1250 RT OGI5754
2018-06-10 Triumph Tiger 1050 JQJ1386
2014-11-14 Triumph Tiger 1050 PZW8010
2020-03-04 BMW R 1250 RT NRL2518
2022-02-08 BMW R 1250 RT RSM0779
2016-09-18 BMW R 1250 RT LPP5080
2018-12-25 BMW R 1250 RT BNR9248
2020-07-05 BMW R 1250 RT INW5596
2019-12-25 Triumph Tiger 1050 FZS1023
2018-12-20 Kawasaki Versys 1000 NAX6047
2014-03-24 Triumph Tiger 1050 IIV4977
2017-02-25 Triumph Tiger 1050 NFP6194
2017-12-11 Kawasaki Versys 1000 NRA7129
2022-10-12 Kawasaki Versys 1000 SWN0995
2019-01-24 Kawasaki Versys 1000 KYI5398
2019-07-06 BMW R 1250 RT YZE6411
2015-08-27 Triumph Tiger 1050 ZYA8326
2015-12-21 Kawasaki Versys 1000 YJV3645
2021-04-03 BM